## 1\. 군집모형 성능 평가

  - **문제 제기**: 마케팅 팀이 K-Means로 나눈 고객 그룹이 얼마나 정확한지, 기존 우수 고객 명단과 얼마나 일치하는지 어떻게 평가할 수 있을까?
  - **필요성**: 군집 분석은 정답 없는 데이터를 탐색하는 도구지만, 때로는 비교할 수 있는 '정답' 혹은 '기준'이 있을 수 있음. 이럴 때 군집화 결과가 기준에 얼마나 부합하는지 객관적인 '점수'로 평가할 필요가 있음.
  - **학습 목표**: 군집 모델이 만들어 낸 그룹이 실제 데이터의 구조나 정답과 얼마나 유사한지를 측정하는 다양한 **성능 평가 지표**를 학습하고, 이를 통해 모델 성능을 비교하거나 최적의 클러스터 개수를 찾는 등 데이터 기반 의사결정을 내림.

-----

## 2\. 핵심 원리 파헤치기 (Deep Dive)

  - **핵심 차이점**: 군집 알고리즘이 부여한 레이블(예: 0, 1, 2)은 실제 레이블(예: '사과', '바나나')과 이름 자체는 관련이 없음.
  - **본질적 질문**: "같은 그룹에 속해야 할 데이터들이 실제로 같은 그룹에 묶였는가?"

### 1\. 랜드 지수 (Rand Index, RI)

  - **`랜드 지수(Rand Index)`**: 두 군집화 결과(모델 예측 U, 실제 정답 V)가 얼마나 일치하는지 평가하는 직관적인 방법.
  - **'사진 분류' 비유**: 당신(U)과 친구(V)가 사진을 분류한 결과를 비교. 무작위로 사진 두 장을 뽑아봄.
      - **a (일치)**: 둘 다 두 사진을 '같은' 폴더에 넣음.
      - **b (일치)**: 둘 다 두 사진을 '다른' 폴더에 넣음.
      - **c, d (불일치)**: 한 명은 '같은' 폴더에, 다른 한 명은 '다른' 폴더에 넣음.
      - 랜드 지수는 전체 사진 쌍 중 **일치한 경우(a+b)의 비율**임.

| 상황 | 설명                               | 쌍 개수 | 의미                 |
| -- | -------------------------------- | ---- | ------------------ |
| a  | 둘 다 ‘같은 폴더’로 분류                  | (a)  | TP (True Positive) |
| b  | 둘 다 ‘다른 폴더’로 분류                  | (b)  | TN (True Negative) |
| c  | 한쪽만 ‘같은 폴더’로, 다른 한쪽은 ‘다른 폴더’로 분류 | (c)  | FN                 |
| d  | 반대 상황                            | (d)  | FP                 |


  - **랜드 지수 공식**:
$$RI = \frac{a+b}{a+b+c+d} = \frac{a+b}{\binom{n}{2}}$$
      - 분모는 전체 데이터에서 2개의 쌍을 뽑는 모든 경우의 수, 분자는 두 군집화 결과가 일치하는 쌍의 개수.
      - 0과 1 사이의 값을 가지며, 1에 가까울수록 완벽하게 일치함을 의미.

### 2\. 조정된 랜드 지수 (Adjusted Rand Index, ARI)

  - **랜드 지수의 함정**: 클러스터 수가 많아지면 우연히 '다른 클러스터에 속하는' 쌍(b)의 수가 늘어나 점수가 높아지는 경향이 있음.
  - **`조정된 랜드 지수(ARI)`**: 이러한 **우연(chance)에 의한 점수 상승을 보정**해주는 지표.
  - **장점**: 두 군집화 결과가 무작위일 경우 0, 완벽하게 일치할 경우 1의 점수를 갖도록 조정됨. 따라서 클러스터 개수가 다른 모델들을 비교할 때 일반 RI보다 훨씬 신뢰할 수 있음.

$$ ARI = \frac{RI - E[RI]}{1 - E[RI]} $$


$$ ARI = \frac{ \mathrm{Index} - \mathbb{E}[\mathrm{Index}] }{ \mathrm{MaxIndex} - \mathbb{E}[\mathrm{Index}] } $$


$$
ARI =
\frac{\sum_{i,j} \binom{n_{ij}}{2} - \frac{\left(\sum_i \binom{a_i}{2}\right)\left(\sum_j \binom{b_j}{2}\right)}{\binom{n}{2}}}
{\frac{1}{2}\left(\sum_i \binom{a_i}{2} + \sum_j \binom{b_j}{2}\right) - \frac{\left(\sum_i \binom{a_i}{2}\right)\left(\sum_j \binom{b_j}{2}\right)}{\binom{n}{2}} }
$$
    

### 3\. 그 외의 주요 평가 지표들

  - **조정된 상호 정보량 (Adjusted Mutual Information, AMI)**: 한 군집화 결과가 다른 군집화 결과에 대해 얼마나 많은 정보를 담고 있는지 측정. 우연에 대해 보정된 값 (0\~1).
  - **포울크스-멜로우 지수 (Fowlkes-Mallows Index, FMI)**: 정밀도와 재현율의 기하 평균. 값이 높을수록 두 군집이 유사함.
  - **Homogeneity, Completeness, V-measure**:
      - **`Homogeneity`**: 각 클러스터가 얼마나 단일 클래스의 데이터로만 구성되었는지 측정 (0\~1).
      - **`Completeness`**: 같은 클래스에 속한 데이터들이 얼마나 같은 클러스터에 모여 있는지 측정 (0\~1).
      - **`V-measure`**: Homogeneity와 Completeness의 조화 평균 (0\~1).

-----

## 3\. 실전 코드 분석 및 적용 (Code Walkthrough & Application)

### 1\. Scikit-learn의 평가 지표 확인하기

In [1]:
# Scikit-learn에서 제공하는 전체 평가 지표 키를 확인합니다.
from sklearn.metrics import get_scorer_names

get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'd2_absolute_error_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_max_error',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'neg_root_mean_squared_log_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 're

- **코드 해설**: 이 목록에는 회귀, 분류, 군집 분석을 위한 다양한 평가 지표가 포함되어 있으며, **`'adjusted_rand_score'`**, **`'homogeneity_score'`** 등이 군집 분석에 사용되는 지표임.

### 2\. 랜드 지수(Rand Index) 직접 구현 및 계산

In [2]:
# 랜드 지수를 직접 계산하는 함수를 정의합니다.
def rand_index(y_true, y_pred):
    n = len(y_true) # 데이터의 총 개수
    a, b = 0, 0 # a와 b 변수 초기화

    # 모든 데이터 쌍(pair)을 순회합니다.
    for i in range(n):
        for j in range(i + 1, n):
            # case 'a': 두 군집화 결과 모두에서 같은 클러스터에 속하는 경우
            if (y_true[i] == y_true[j]) & (y_pred[i] == y_pred[j]):
                a += 1
            # case 'b': 두 군집화 결과 모두에서 다른 클러스터에 속하는 경우
            elif (y_true[i] != y_true[j]) & (y_pred[i] != y_pred[j]):
                b += 1
            else:
                pass # 그 외의 경우는 세지 않음

    # 랜드 지수 공식을 적용하여 RI를 계산합니다.
    RI = (a + b) / (n * (n - 1) / 2)
    return RI

In [3]:
# 필요한 라이브러리를 임포트합니다.
import seaborn as sns
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder

# iris 데이터셋을 로드합니다.
iris = sns.load_dataset("iris")

# 특성 데이터(X)와 정답 레이블(y)을 분리합니다.
iris_X = iris.iloc[:, :-1]
iris_y = iris.species

# K-Means 모델을 생성하고 학습시킵니다 (클러스터 수 k=3).
iris_cluster_model = KMeans(n_clusters=3, random_state=1)
iris_cluster_model.fit(iris_X)

# K-Means가 예측한 클러스터 레이블(0, 1, 2)을 실제 품종 레이블 순서와 맞추기 위해 재조정합니다.
# 주의: 이 과정은 실제 분석에서는 필요 없을 수 있으며, 어떤 클러스터가 어떤 품종에 해당하는지 알 수 없기 때문입니다.
# 여기서는 평가를 위해 임의로 맞추는 과정입니다.
pred = np.choose(iris_cluster_model.labels_, [1, 0, 2])

# 문자열로 된 실제 품종 레이블을 숫자(0, 1, 2)로 변환합니다.
le = LabelEncoder()
le.fit(iris.species)
predict = le.inverse_transform(pred) # 숫자 레이블을 다시 품종 이름으로 변환

# 직접 구현한 랜드 지수 함수를 호출하여 점수를 확인합니다.
rand_index(y_true=iris.species, y_pred=predict)

0.8737360178970918

- **코드 해설**: `iris` 데이터의 실제 품종이 3개이므로 K-Means의 \*\*`n_clusters`\*\*를 3으로 설정함. **`rand_index`** 계산 결과 약 0.88로, K-Means 군집화 결과가 실제 품종 분류와 상당히 일치함을 의미함.

### 3\. 클러스터 개수를 바꿔서 비교하기

In [4]:
# K-Means 모델을 클러스터 2개로 다시 학습시킵니다.
iris_cluster_model = KMeans(n_clusters=2, random_state=1)
iris_cluster_model.fit(iris_X)

# 정답 레이블을 생성합니다: setosa는 1, 나머지(versicolor, virginica)는 0으로 설정
y_true = [1]*50 + [0]*100

# 랜드 지수를 계산합니다.
rand_index(y_true, iris_cluster_model.labels_)

0.9605369127516779

- **코드 해설**: 클러스터를 2개로 설정했을 때 랜드 지수가 약 0.96으로 더 높게 나왔음. 이는 RI가 클러스터 수에 영향을 받으므로 완벽한 비교는 아님.

### 4\. Scikit-learn의 다양한 평가 지표 사용하기

  - 우연을 보정한 `Adjusted Rand Index (ARI)`와 다른 지표들을 계산. 이 함수들은 레이블을 수동으로 재조정할 필요가 없어 편리함.

<!-- end list -->

In [5]:
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, fowlkes_mallows_score, homogeneity_score, completeness_score, v_measure_score

# 이전 단계에서 k=3으로 군집화한 예측 결과 'predict'를 사용합니다.

# 조정된 랜드 지수 (ARI)
ari_score = adjusted_rand_score(labels_true=iris.species, labels_pred=predict)
print(f"Adjusted Rand Index: {ari_score}")

# 조정된 상호 정보량 (AMI)
ami_score = adjusted_mutual_info_score(labels_true=iris.species, labels_pred=predict)
print(f"Adjusted Mutual Info Score: {ami_score}")

# 포울크스-멜로우 지수 (FMI)
fmi_score = fowlkes_mallows_score(labels_true=iris.species, labels_pred=predict)
print(f"Fowlkes-Mallows Score: {fmi_score}")

# Homogeneity, Completeness, V-measure
h_score = homogeneity_score(labels_true=iris.species, labels_pred=predict)
c_score = completeness_score(labels_true=iris.species, labels_pred=predict)
v_score = v_measure_score(labels_true=iris.species, labels_pred=predict)
print(f"Homogeneity Score: {h_score}")
print(f"Completeness Score: {c_score}")
print(f"V-measure Score: {v_score}")

Adjusted Rand Index: 0.7163421126838476
Adjusted Mutual Info Score: 0.7386548254402864
Fowlkes-Mallows Score: 0.8112427991975698
Homogeneity Score: 0.7364192881252849
Completeness Score: 0.7474865805095324
V-measure Score: 0.7419116631817836


- **코드 해설**: 다양한 지표를 통해 군집 모델의 성능을 다각도로 평가할 수 있음. **ARI 점수**는 약 0.73으로, 무작위 추측(0)보다는 훨씬 좋지만 완벽한(1) 수준은 아님을 보여줌. 여러 지표를 함께 확인하여 모델 성능을 종합적으로 판단하는 것이 좋음.

-----

## 4\. 핵심 요약 (Key Takeaways)

  - **비즈니스 관점**: 군집 분석 결과를 기존의 '정답' 데이터와 비교하여, 군집 모델이 **비즈니스적으로 의미 있는 패턴**을 찾아냈는지 객관적으로 검증 가능함.
  - **기술 관점**: 단순 \*\*`랜드 지수(RI)`\*\*는 클러스터 개수에 따라 점수가 왜곡될 수 있으므로, 우연에 의한 효과를 보정한 \*\*`조정된 랜드 지수(ARI)`\*\*나 \*\*`조정된 상호 정보량(AMI)`\*\*을 사용하는 것이 훨씬 신뢰성이 높음.
  - **해석 관점**: 군집 성능 평가 지표는 대부분 0과 1 사이의 값을 가짐. 점수가 **1에 가까울수록** 군집 결과가 실제 레이블과 완벽하게 일치하며, **0에 가까울수록** 무작위 분류와 차이가 없음을 의미함.